# Introduction

Playlists appeal to many music listeners today, giving them the ability to create songs for specific situations. After all, the music you listen to while working out or attending a party is probably vastly different from the music you’d listen to while studying for a test or playing frisbee with friends. Unfortunately, making playlists is a chore. Wouldn’t it be easier if you could just have playlists made for you, based on your current music listening history and likes?

Spotify currently attempts to do this with daily mixes that are a combination of your likes and new music, but separated into different genres(i.e. daily mix of your rap vs. classic rock favorites). And they also have specific playlists for different situations that aren't based on your tastes, i.e. for working out or studying. But Spotify and music listeners in general lack a tool that combines these abilities, a tool that makes playlists for different situations based on your music preferences. 

This was the inspiration for my project; creating a model that would be able to analyze user music preferences and create different playlists for different situations. This model was based off freely available data found on Kaggle covering the characteristics of nearly 60,000 unique songs since 1929. The song features included in the dataset that are used in this model are song name, artist, popularity, duration, danceability, energy, instrumentalness, acousticness, speechiness, and valence. The reason these specific characteristics were chosen will be explained in the mathematical model section, but the dataset includes  more features such as mode, key, liveness, explicit, and release date. 

The rest of the report describes the three different mathematical models used to generate playlists, one for high energy situations like working out and one for calmer situations like studying. These first two models generate playlists with the most popular songs that fit the situation. This model doesn’t account for user preferences. The third model uses a technique called regularization that attempts to match as many songs in the generated playlist to songs liked by the user while also maximizing the popularity of the songs. 


# Mathematical Model

The first two mathematical models aim to create optimal playlists for high energy and low energy situations, disregarding user preferences. They both have the same objective functions and decision variables, but have varying constraints. For simplicity sake, we will refer to the high energy playlist as model 1 and the low energy playlist as model 2. Objective funtions for first two models:

$$max\sum_{i=1}^N X_i * P_i $$
where N = total # songs in data, X_i is a binary variable that = 1 if song i chosen to be in optimal playlist, 0 if not. P_i represents the popularity rating of song i. 

Decision Variables, both nonnegative are $$X_i, P_i$$

Constraints:

Models 1 and 2 have the constraint that the total length of the optimal playlist must be between 1.8e6 and 3.6e6 ms, or approximately 30 to 60 minutes long. Each individual song in the optimal playlist also had to have a length of atleast 1.5 minutes. Note that all the variables in the constraints besides duration have values between 0 and 1, used to indicate the amount that a song fits a certain music criteria, i.e. danceability = 1 means a song is very danceable. 

Model 1 aims to create a high energy playlist with 3 key constraints; for each song, the danceability and tempo  rating are >= 0.66, and the energy rating is >= 0.75. The specific lower bounds of these constraints were chosen by analyzing the statistical distribution of ratings for songs in the Spotify database, which is provided on the Spotify website. For example, the danceability rating has an approximately normal distribution, which is why 0.66 was chosen as the lower bound,  as we wanted songs that are highly danceable, while the energy rating has an approximately right-skewed distribution, which is why 0.75 was chosen as the lower bound instead. 

There is a second version of model 1 that incorporates a fourth constraint, that the valence rating is >= 0.5, meaning the song has more positive tones. Many of the songs outputted by the first model have a much darker tone, and in the spirit of a workout/party playlist, the valence constraint was added, potentially making this a more optimal playlist. However, it’s unclear whether more positive music corresponds to “better” workout or party music. 

Model 2 aims to create a low energy playlist with 7 key constraints. For each song, the acousticness rating is >= 0.33, instrumentalness >= 0.2, danceability <= 0.5, loudness <= 0.66, energy <= 0.66, speechiness <= 0.33, and tempo <= 0.5. The distributions of the values of these variables were again analyzed to determine the lower bounds of the constraints. 

For example, acousticness and speechiness are left-skewed, while the majority of values in instrumentalism are less than 0.2, which is why this was chosen as the lower bounds. Energy and loudness are right skewed, but 0.66 was chosen as the lower bound instead of 0.75 because we’re attempting to emphasize getting songs in the optimal playlist with much lower values for these variables. 0.5 was chosen as the lower bound for danceability and tempo because of their relatively normal distributions. Like model 1, a second version of model 2 was made to incorporate the valence characteristic, in order to create a low energy playlist with positive tones that could be used for meditating. 

Model 3 aims to maximize popularity of a playlist between 30 minutes and 1 hour (only constraint in the model) while also matching the songs in the playlist to the given user songs, which represent their music preferences. The optimization function for this model is:

$$min\sum_{i=1}^N X_i * P_i + λ*(Y_i - X_i)^2 $$

Where λ represents the degree to which we’re matching the songs, i.e. λ = 0 means user songs aren’t considered, while λ = 1 means every song in our new optimal playlist is also from the given user songs. 
And where N represents the total # songs in the data, X_i represents if a song i is chosen to be in the optimal playlist, and Y_i represents if song i from our complete dataset is in the songs given by the user, i.e. 1 if it is, 0 otherwise. Y_i is an additonal decision variable in model 3. 

Assumptions made in mathematical models:

While the lower bounds for all the constraints in the mathematical models were determined by analyzing the statistical distributions of different song characteristics, there are still assumptions made about which values of song features correspond to which kind of playlist. For instance, I made the assumption that the most important characteristics for the workout playlist were the danceability, energy, and tempo ratings. Likewise, I made assumptions in determining that acousticness, instrumentalness, danceability, loudness, energy, speechiness, and tempo were the key characteristics for a low-energy playlist used for studying or meditating. 

To make an optimal playlist, popularity of each song in the playlist was maximized. Using popularity as the key feature in determining an optimal song is an assumption, as this may not be the best way to determine if a song is best for a given situation, i.e. working out versus studying. The models were designed so that the characteristics of different types playlists were matched through the constraints, while maximizing popularity would create a playlist with songs that most listeners have heard before and would enjoy in a greater capacity. 

For the first two models, each song in the playlist had to be at least 1.5 minutes. An assumption was made that most listeners would prefer to listen to songs of at least this length. For all three models, an assumption was made in that the optimal playlist had a length of 30 to 60 minutes. Determining optimal song and playlist length requires more research. 

A quick description of all the variables used in the constraints of the model is given below, except for the variable duration, which simply represents the length of the track in milliseconds. 
* Acousticness: Confidence measure from 0.0 to 1.0 of whether track is acoustic, i.e. 1.0 represents high confidence track is acoustic. 
* Danceability: Describes how suitable  track is for dancing based on combo of music elements including tempo, rhythm, stability, beat strength, and overall regularity. 
* Energy: Perceptual measure of intensity and activity (i.e. fast/loud/noisy). Features contributing to energy are dynamic range, perceived loudness, timbre, obset rate, and general entropy. 
* Instrumentalness: Predicts whether a track contains no vocals i.e. above 0.5 is instrumental trucks, no vocal content is 1.0. Majority of values are less than 0.2, which is why this has a unique lower bound. 
* Loudness: In decibels, values averaged across entire track, psychological correlate of amplitude.
* Speechiness: Detects presence of spoken words in track, i.e. talk show, audio book, poetry is 1.0. 
* Tempo: Overall estimated tempo in beats per minute
* Valence: Musical positiveness conveyed by track, i.e. 1.0 is cheerful vs 0.0 is depressed. 
 
Finally, this is a mixed integer program as we have variables that must have integer values i.e. Xi = 1 or 0 if the song is chosen to be in optimal playlist. But we also have variables that don't have integer values, like some of the variables in the constraints that have fractional values between 0 and 1, i.e. danceability can be 0.66. 
 

# Solution

In [1]:
using DataFrames, CSV, NamedArrays

In [2]:
# Transform dataframe to relevant named arrays

df = CSV.read("total_data.csv", header = true, delim = ',')
#features = propertynames(df)

songs = convert(Array,df[1:end,2]) 

art = convert(Array, df[1:end,3])
artist = Dict(zip(songs, art))

pop = convert(Array,df[1:end,4]) 
popularity = Dict(zip(songs, pop)) 

dur = convert(Array,df[1:end,5])
duration = Dict(zip(songs,dur))

dance = convert(Array,df[1:end,6]) 
danceability = Dict(zip(songs,dance))

en = convert(Array,df[1:end,7]) 
energy = Dict(zip(songs,en))

tem = convert(Array,df[1:end,8]) 
tempo = Dict(zip(songs,en))

loud = convert(Array,df[1:end,9]) 
loudness = Dict(zip(songs,loud))

val = convert(Array,df[1:end,10]) 
valence = Dict(zip(songs,val))

ac = convert(Array,df[1:end,11]) 
acousticness = Dict(zip(songs,ac))

ins = convert(Array,df[1:end,12]) 
instrumentalness = Dict(zip(songs,ins))

speech = convert(Array,df[1:end,13])
speechiness = Dict(zip(songs,speech))

exp = convert(Array,df[1:end,14])
explicit = Dict(zip(songs,exp))

#using NamedArrays
#song_feature_matrix = convert(Matrix,df[1:end,2:end])
#song_feature_array = NamedArray(song_feature_matrix, (songs, features), ("songs", "features"))

┌ Warning: `CSV.read(input; kw...)` is deprecated in favor of `DataFrame!(CSV.File(input; kw...))`
└ @ CSV /Users/niyaz/.julia/packages/CSV/URGyF/src/CSV.jl:40


Dict{String,Int64} with 54102 entries:
  "Preso"                                                                   => 0
  "Heywood's Bounce"                                                        => 0
  "Deafinitely"                                                             => 0
  "Piano Sonata No. 10 in C Major, K. 330: III. Allegretto"                 => 0
  "Mi Ritmo Te Llama"                                                       => 0
  "El Tecolote"                                                             => 0
  "Freakish"                                                                => 0
  "Good Times (Let The Good Times Roll)"                                    => 0
  "Samadhite Mor Phul Chharate Ke Go Ele"                                   => 0
  "Soft Skin"                                                               => 0
  "Piano Sonata No.14 in C minor, K.457: 3. Allegro assai"                  => 0
  "Verdi : Un ballo in maschera : Act 1 - Quadro I \"Posa in pace\" [C

In [3]:
# Model for generating high-energy playlists for working out/partying

using JuMP, Gurobi

m = Model(Gurobi.Optimizer) 
@variable(m, x[songs], Bin)# create blank list of songs, becomes optimal playlist 

@objective(m, Max, sum(x[i]*popularity[i] for i in songs)) #maximize the sum of popularity scores among songs

@constraint(m, length, sum(duration[i]*x[i] for i in songs) <= 3.6e6)
@constraint(m, length2, sum(duration[i]*x[i] for i in songs) >= 1.8e6) # playlist between 30 and 60 minutes
@constraint(m, length3[i in songs], duration[i]*x[i] >= 90000 * x[i]) # individual song must be 1.5 minutes

@constraint(m, dance1[i in songs], x[i] * danceability[i] >= 0.66 * x[i]) # because of normal distribution
@constraint(m, energy1[i in songs], x[i] * energy[i] >= 0.75 * x[i]) # right skew 
@constraint(m, tempo1[i in songs], x[i] * tempo[i] >= 0.66 * x[i])


optimize!(m)
println(objective_value(m))
println("")
for i in songs
    if value(x[i]) != 0
        println(i, " by ", artist[i], "  ", round(duration[i] * 1.66667e-5,digits = 1), " minutes ", popularity[i], " popularity score")
    end
end

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 216410 rows, 54102 columns and 324426 nonzeros
Model fingerprint: 0x467e1c49
Variable types: 0 continuous, 54102 integer (54102 binary)
Coefficient statistics:
  Matrix range     [1e-03, 5e+06]
  Objective range  [1e+00, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+06, 4e+06]
Presolve removed 216408 rows and 51333 columns
Presolve time: 0.16s
Presolved: 2 rows, 2769 columns, 5538 nonzeros
Variable types: 0 continuous, 2769 integer (2766 binary)
Found heuristic solution: objective 806.0000000

Root relaxation: objective 1.780422e+03, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1780.42237    0    1  806.00000 1780.42237   121%     -    0s
H    0     0     

In [4]:
# Model for generating high-energy playlists for working out/partying

using JuMP, Gurobi

m = Model(Gurobi.Optimizer) 
@variable(m, x[songs], Bin)# create blank list of songs, becomes optimal playlist 

@objective(m, Max, sum(x[i]*popularity[i] for i in songs)) #maximize the sum of popularity scores among songs

@constraint(m, length, sum(duration[i]*x[i] for i in songs) <= 3.6e6)
@constraint(m, length2, sum(duration[i]*x[i] for i in songs) >= 1.8e6) # playlist between 30 and 60 minutes
@constraint(m, length3[i in songs], duration[i]*x[i] >= 90000 * x[i]) # individual song must be 1.5 minutes

@constraint(m, dance1[i in songs], x[i] * danceability[i] >= 0.66 * x[i]) # because of normal distribution
@constraint(m, energy1[i in songs], x[i] * energy[i] >= 0.75 * x[i]) # right skew 
@constraint(m, valence1[i in songs], x[i] * valence[i] >= 0.5 * x[i]) #positive music
@constraint(m, tempo1[i in songs], x[i] * tempo[i] >= 0.66 * x[i])


optimize!(m)
println(objective_value(m))
println("")
for i in songs
    if value(x[i]) != 0
        println(i, " by ", artist[i], "  ", round(duration[i] * 1.66667e-5,digits = 1), " minutes ", popularity[i], " popularity score")
    end
end

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 270512 rows, 54102 columns and 378476 nonzeros
Model fingerprint: 0x3aa9a4f9
Variable types: 0 continuous, 54102 integer (54102 binary)
Coefficient statistics:
  Matrix range     [1e-03, 5e+06]
  Objective range  [1e+00, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+06, 4e+06]
Presolve removed 270510 rows and 51671 columns
Presolve time: 0.18s
Presolved: 2 rows, 2431 columns, 4862 nonzeros
Variable types: 0 continuous, 2431 integer (2428 binary)
Found heuristic solution: objective 755.0000000

Root relaxation: objective 1.688133e+03, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1688.13253    0    1  755.00000 1688.13253   124%     -    0s
H    0     0     

In [16]:
# Model for generating low-energy playlists for studying

using JuMP, Gurobi

m = Model(Gurobi.Optimizer) 
@variable(m, x[songs], Bin)# create blank list of songs, becomes optimal playlist 

@objective(m, Max, sum(x[i]*popularity[i] for i in songs)) #maximize the sum of popularity scores among songs



@constraint(m, length, sum(duration[i]*x[i] for i in songs) <= 3.6e6)
@constraint(m, length2, sum(duration[i]*x[i] for i in songs) >= 1.8e6) # playlist between 30 and 60 minutes
@constraint(m, length3[i in songs], duration[i]*x[i] >= 90000 * x[i]) # individual song must be 1.5 minutes


@constraint(m, acoustic1[i in songs], x[i] * acousticness[i] >= 0.33 * x[i]) # because of left skew
@constraint(m, inst1[i in songs], x[i] * instrumentalness[i] >= 0.2 * x[i]) # majority of values < 0.2
@constraint(m, dance2[i in songs], x[i] * danceability[i] <= 0.66 * x[i])
@constraint(m, loudness2[i in songs], x[i] * loudness[i] <= 0.75 * x[i])
@constraint(m, energy2[i in songs], x[i] * energy[i] <= 0.75 * x[i])
@constraint(m, speech[i in songs], x[i] * speechiness[i] <= 0.33 * x[i])
@constraint(m, tempo2[i in songs], x[i] * tempo[i] <= 0.66 * x[i])



optimize!(m)
println(objective_value(m))
println("")
for i in songs
    if value(x[i]) != 0
        println(i, "  by ", artist[i], "  ", round(duration[i] * 1.66667e-5,digits = 1), " minutes  ", popularity[i], " popularity score")
    end
end

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 432818 rows, 54102 columns and 540786 nonzeros
Model fingerprint: 0xe84f0a95
Variable types: 0 continuous, 54102 integer (54102 binary)
Coefficient statistics:
  Matrix range     [1e-03, 5e+06]
  Objective range  [1e+00, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+06, 4e+06]
Presolve removed 432816 rows and 47868 columns
Presolve time: 0.28s
Presolved: 2 rows, 6234 columns, 12468 nonzeros
Variable types: 0 continuous, 6234 integer (6061 binary)
Found heuristic solution: objective 359.0000000
Found heuristic solution: objective 672.0000000

Root relaxation: objective 1.986746e+03, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1986.74562    0    1  672.0000

In [19]:
# Model for generating low-energy playlists for studying

using JuMP, Gurobi

m = Model(Gurobi.Optimizer) 
@variable(m, x[songs], Bin)# create blank list of songs, becomes optimal playlist 

@objective(m, Max, sum(x[i]*popularity[i] for i in songs)) #maximize the sum of popularity scores among songs



@constraint(m, length, sum(duration[i]*x[i] for i in songs) <= 3.6e6)
@constraint(m, length2, sum(duration[i]*x[i] for i in songs) >= 1.8e6) # playlist between 30 and 60 minutes
@constraint(m, length3[i in songs], duration[i]*x[i] >= 90000 * x[i]) # individual song must be 1.5 minutes


@constraint(m, acoustic1[i in songs], x[i] * acousticness[i] >= 0.33 * x[i]) # because of left skew
@constraint(m, inst1[i in songs], x[i] * instrumentalness[i] >= 0.2 * x[i]) # majority of values < 0.2
@constraint(m, dance2[i in songs], x[i] * danceability[i] <= 0.5 * x[i])
@constraint(m, loudness2[i in songs], x[i] * loudness[i] <= 0.66 * x[i])
@constraint(m, energy2[i in songs], x[i] * energy[i] <= 0.66 * x[i])
@constraint(m, speech[i in songs], x[i] * speechiness[i] <= 0.33 * x[i])
@constraint(m, tempo2[i in songs], x[i] * tempo[i] <= 0.5 * x[i])
@constraint(m, valence2[i in songs], x[i] * valence[i] >= 0.5 * x[i]) #positive tones



optimize!(m)
println(objective_value(m))
println("")
for i in songs
    if value(x[i]) != 0
        println(i, "  by ", artist[i], "  ", round(duration[i] * 1.66667e-5,digits = 1), " minutes  ", popularity[i], " popularity score")
    end
end

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 486920 rows, 54102 columns and 594797 nonzeros
Model fingerprint: 0xc803e4d2
Variable types: 0 continuous, 54102 integer (54102 binary)
Coefficient statistics:
  Matrix range     [1e-03, 5e+06]
  Objective range  [1e+00, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+06, 4e+06]
Presolve removed 486918 rows and 53201 columns
Presolve time: 0.47s
Presolved: 2 rows, 901 columns, 1802 nonzeros
Variable types: 0 continuous, 901 integer (879 binary)
Found heuristic solution: objective 956.0000000

Root relaxation: objective 1.082263e+03, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1082.26331    0    1  956.00000 1082.26331  13.2%     -    0s
H    0     0        

In [7]:
#reads in user songs and returns an optimal playlist given a lambda value for solve opt function. 

using JuMP, Gurobi

df2 = CSV.read("user_data.csv", header = true, delim = ',')
user_songs = convert(Array,df2[1:end,2]) 
z = zeros(Int32,54102)
a = size(songs)

for i in 1:a[1]
    if songs[i] in user_songs
        z[i] = 1
    end
end
     
user_songs_bin = Dict(zip(songs,z))

function solveOpt(λ)

    m = Model(Gurobi.Optimizer)

    @variable(m, alg_songs[songs],Bin) # songs produced by algorithm 

    @objective(m, Min, sum(alg_songs[i]*popularity[i] for i in songs)
        +
        λ*sum((user_songs_bin[i] - alg_songs[i])^2 for i in songs)
    )
    
    @constraint(m, length, sum(duration[i]*alg_songs[i] for i in songs) <= 3.6e6)
    @constraint(m, length2, sum(duration[i]*alg_songs[i] for i in songs) >= 1.8e6)


    optimize!(m)

    y1 = value(sum(alg_songs[i]*popularity[i] for i in songs))
    
    y2 = value(sum((user_songs_bin[i] - alg_songs[i])^2 for i in songs))
    
    xopt = value.(alg_songs)

    c = 0
    for i in xopt
        c = c+1
        if i > 0
            println(songs[c])
        end
    end
    #return (y1,y2,xopt)

end

;

┌ Warning: `CSV.read(input; kw...)` is deprecated in favor of `DataFrame!(CSV.File(input; kw...))`
└ @ CSV /Users/niyaz/.julia/packages/CSV/URGyF/src/CSV.jl:40


In [8]:
print(solveOpt(0))

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 2 rows, 54102 columns and 108204 nonzeros
Model fingerprint: 0x95e566d9
Variable types: 0 continuous, 54102 integer (54102 binary)
Coefficient statistics:
  Matrix range     [6e+03, 5e+06]
  Objective range  [1e+00, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+06, 4e+06]
Found heuristic solution: objective 152.0000000
Presolve removed 0 rows and 2476 columns
Presolve time: 0.53s
Presolved: 2 rows, 51626 columns, 103252 nonzeros
Variable types: 0 continuous, 51626 integer (49594 binary)

Starting sifting (using dual simplex for sub-problems)...

    Iter     Pivots    Primal Obj      Dual Obj        Time
       0          0     infinity      0.0000000e+00      1s
       1          1   1.4388738e-01   1.1484834e-01      1s

Sifting complete


Root relaxation: objective 0.000000e+00, 3 iterations, 0.06

In [9]:
print(solveOpt(0.1))

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 2 rows, 54102 columns and 108204 nonzeros
Model fingerprint: 0x70769e02
Model has 54102 quadratic objective terms
Variable types: 0 continuous, 54102 integer (54102 binary)
Coefficient statistics:
  Matrix range     [6e+03, 5e+06]
  Objective range  [2e-01, 8e+01]
  QObjective range [2e-01, 2e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+06, 4e+06]
Found heuristic solution: objective 225.5000000
Presolve removed 0 rows and 2472 columns
Presolve time: 0.56s
Presolved: 2 rows, 51630 columns, 103260 nonzeros
Variable types: 0 continuous, 51630 integer (49602 binary)

Starting sifting (using dual simplex for sub-problems)...

    Iter     Pivots    Primal Obj      Dual Obj        Time
       0          0     infinity      1.2900000e+01      1s

Sifting complete


Root relaxation: objective 1.296405e+01, 1 

In [10]:
print(solveOpt(0.3))

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 2 rows, 54102 columns and 108204 nonzeros
Model fingerprint: 0xb48b29d0
Model has 54102 quadratic objective terms
Variable types: 0 continuous, 54102 integer (54102 binary)
Coefficient statistics:
  Matrix range     [6e+03, 5e+06]
  Objective range  [4e-01, 8e+01]
  QObjective range [6e-01, 6e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+06, 4e+06]
Found heuristic solution: objective 254.5000000
Presolve removed 0 rows and 2472 columns
Presolve time: 0.55s
Presolved: 2 rows, 51630 columns, 103260 nonzeros
Variable types: 0 continuous, 51630 integer (49602 binary)

Starting sifting (using dual simplex for sub-problems)...

    Iter     Pivots    Primal Obj      Dual Obj        Time
       0          0     infinity      3.8700000e+01      1s

Sifting complete


Root relaxation: objective 3.889214e+01, 1 

In [11]:
print(solveOpt(0.5))

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 2 rows, 54102 columns and 108204 nonzeros
Model fingerprint: 0x414792c4
Model has 54102 quadratic objective terms
Variable types: 0 continuous, 54102 integer (54102 binary)
Coefficient statistics:
  Matrix range     [6e+03, 5e+06]
  Objective range  [1e+00, 8e+01]
  QObjective range [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+06, 4e+06]
Found heuristic solution: objective 283.5000000
Presolve removed 0 rows and 2480 columns
Presolve time: 0.46s
Presolved: 2 rows, 51622 columns, 103244 nonzeros
Variable types: 0 continuous, 51622 integer (49586 binary)

Starting sifting (using dual simplex for sub-problems)...

    Iter     Pivots    Primal Obj      Dual Obj        Time
       0          0     infinity      6.4500000e+01      1s

Sifting complete


Root relaxation: objective 6.482023e+01, 1 

In [12]:
print(solveOpt(0.7))

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 2 rows, 54102 columns and 108204 nonzeros
Model fingerprint: 0xcb135901
Model has 54102 quadratic objective terms
Variable types: 0 continuous, 54102 integer (54102 binary)
Coefficient statistics:
  Matrix range     [6e+03, 5e+06]
  Objective range  [4e-01, 8e+01]
  QObjective range [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+06, 4e+06]
Found heuristic solution: objective 312.5000000
Presolve removed 0 rows and 2472 columns
Presolve time: 0.57s
Presolved: 2 rows, 51630 columns, 103260 nonzeros
Variable types: 0 continuous, 51630 integer (49602 binary)

Starting sifting (using dual simplex for sub-problems)...

    Iter     Pivots    Primal Obj      Dual Obj        Time
       0          0     infinity      9.0300000e+01      1s

Sifting complete


Root relaxation: objective 9.074832e+01, 1 

In [13]:
print(solveOpt(0.9))

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 2 rows, 54102 columns and 108204 nonzeros
Model fingerprint: 0x330a5ee3
Model has 54102 quadratic objective terms
Variable types: 0 continuous, 54102 integer (54102 binary)
Coefficient statistics:
  Matrix range     [6e+03, 5e+06]
  Objective range  [8e-01, 8e+01]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+06, 4e+06]
Found heuristic solution: objective 341.5000000
Presolve removed 0 rows and 2472 columns
Presolve time: 0.42s
Presolved: 2 rows, 51630 columns, 103260 nonzeros
Variable types: 0 continuous, 51630 integer (49602 binary)

Starting sifting (using dual simplex for sub-problems)...

    Iter     Pivots    Primal Obj      Dual Obj        Time
       0          0     infinity      1.1610000e+02      0s

Sifting complete


Root relaxation: objective 1.166764e+02, 1 

In [14]:
print(solveOpt(1))

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 2 rows, 54102 columns and 108204 nonzeros
Model fingerprint: 0xcff4f83a
Model has 54102 quadratic objective terms
Variable types: 0 continuous, 54102 integer (54102 binary)
Coefficient statistics:
  Matrix range     [6e+03, 5e+06]
  Objective range  [1e+00, 8e+01]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+06, 4e+06]
Found heuristic solution: objective 356.0000000
Presolve removed 0 rows and 2473 columns
Presolve time: 0.44s
Presolved: 2 rows, 51629 columns, 103258 nonzeros
Variable types: 0 continuous, 51629 integer (49600 binary)

Starting sifting (using dual simplex for sub-problems)...

    Iter     Pivots    Primal Obj      Dual Obj        Time
       0          0     infinity      1.2900000e+02      1s

Sifting complete


Root relaxation: objective 1.295829e+02, 1 

# Results and Discussion

It's difficult to judge the accuracy of the playlists created for the different optimal situations because of their subjective nature. Moreover, the songs that constitute a playlist for working out or studying vary for each music listener. In order to get a truly accurate judgement of these different playlists, the opinions of many different music listeners must be gathered. This isn’t possible within the timeframe of this optimization class, so I simply listened to both playlists and judged whether they were optimal for the given situation. 

I was pleasantly surprised at the quality of the playlists generated by this model. The songs in the workout playlist without the valence constraint have many high-energy songs that would be perfect for the workout setting. Many of the songs are also highly danceable and motivating. The constraints for this model created a playlist that fits the workout setting. Maximizing tge popularity characteristic also resulted in many songs from both playlists being songs that I recognized. A previously unrealized benefit of maximizing popularity is that people listening to these generated playlists will listen to music with titles and artists that they will recognize, potentially resulting in greater fulfillment for the listener as they want to listen to songs that their peers are listening to as well. Another benefit of generating the playlists this way is that it doesn’t exclude songs based on language or genre. It was refreshing to hear a playlist with songs from a diverse array of genres and artists. This not only reduced the montonity of the playlist, but also led to greater personal exploration of these new songs from music genres which I had lacked exposure to. For instance, some of the songs in the workout playlist were spanish songs that were quite fun and potentially ideal for a party or workout. 

However, there were a few songs within this optimal playlist that may be bad songs to have in a workout playlist. While these songs may not fit the workout or party environment, they do still have a high tempo and danceability. But they lack the “epic” feel apparent in many other songs in this playlist. As mentioned before, it’s interesting to note that many of the songs in this first workout playlist contained a much darker tone. The high tempo and energy characteristics seem to correlate more with darker music naturally. Thus, I created a second workout playlist with an additional constraint that valence >= 0.5, meaning the song has more positive tones. Roughly half of the songs in this new playlist were in the old playlist, but the general happier tone of this playlist might appeal more to the average music listener. 

I believe the quality of the low energy playlists, used for studying and meditation, is even higher in terms of fitting the given situation. This is because there were more constraints included in the second model, as I believed there were more characteristics in the dataset that could represent a low energy song. For instance, characteristics like acousticness, speechiness, and instrumentalism were more relevant for this model. A song with a larger presence of instrument and acoustic sounds would be more relevant for studying, in addition to a song with a lack of vocals. In the study playlist, many of the songs had a significant instrumental presence, often the piano, which is often a key characteristic for good study music. Many of these songs lacked vocals and their slower pace also means calmer music that allows for greater focus. Moreover, by adding the valence constraint for a second version of the low energy playlist, we create an optimal playlist for meditation, containing positive, low-energy music. However, it's interesting to note that adding the valence constraint for the low energy music model created higher energy music overall in this optimal playlist. Positive music may correlate higher with higher tempo music, and perhaps should be restricted if one wants to create an ideal study playlist. 

In order to improve these playlists in the future, greater research into what characteristics constitute different kinds of optimal playlists, the length of an optimal playlist, and what types of playlists music listeners most desire, would greatly improve the usability of these playlists. 

Because of the nature of these results, there aren't many types of visuals that can be used to depict the success of this model. For the first two models, I am sharing the link to the playlists created so people can judge on their accuracy on their own:

Model 1, Workout Version 1: https://open.spotify.com/playlist/1PaaDT3Ns0in3TNSmTNhoP?si=_O5ep6BcR7eWUC6ZZayUzg

Model 1, Workout Version 2(with valence): https://open.spotify.com/playlist/53FOmHEbFPF3iqBXkhHCZF?si=L6WTIHNbQ4C4ZA4iP_43vQ

Model 2, Study Version 1: https://open.spotify.com/playlist/7i4JbiM0rloDYurlBFMbvI?si=r0pOOT0YT9KQFZnSRoPepw

Model 2, Study Version 2 (with valence, referred to as meditation playlist in report): https://open.spotify.com/playlist/0bjzfEKHTsr1dyb0XNF1Sp?si=wRL3mTZsS0mj3NW7n_jmlg


For the last model, which utilizes regularization to create playlists with maximum popularity while also matching user preferences, we can take a sample set of user songs and check the output of the optimal playlist, changing the lambda values from low to high to see the effect on the optimal playlist. The songs starting with the letter "Y" are from user songs. 

Optimal Playlist for Lambda = 0

Ponchielli : La Gioconda : Act 2 "Sia gloria ai canti" [Enzo, Chorus]
Der Zigeunerbaron, Operetta in 3 Acts - Act 2: "Wer uns getraut"
Часть 38.2 - По ком звонит колокол
Idi Manchi Samayamu
Die Walküre (2007 - Remaster), Act III, Dritte Szene: Deinen leichten Sinn lass dich denn leiten
Tou Kitsou I Mana
Sandmännchen
Pause Track
Ah ! scellerata ! da tua madre
Anthoula
Mozart: Symphony No. 41 in C Major, K. 551 "Jupiter": III. Menuetto. Allegretto


Optimal Playlist for Lambda = 0.1/0.3/0.5/0.7/0.9

The Great Donivitch Kayeoff
You Can Always Count on Me
Yellow Gal
Y de ahi
Yira Yira - Remasterizado
Yankee Volunteer


Optimal Playlist for Lambda = 1

You Can Always Count on Me
You Are Not My First Love
Yellow Gal
Y de ahi
Ghana El Rabea
Yira Yira - Remasterizado
Yankee Volunteer


Thus, we can see that the regularization method is somewhat effective at generating optimal playlists based on user preferences. When lambda values are lower, there are more songs from the larger Spotify database, and when lambda is high, the songs are almost solely from the given user songs. However, it is concerning that once the lambda value reaches 0.1 and keeps rising, the songs in the optimal playlist stay the same. Two of the songs in the optimal playlists are from the Spotify database, while the rest match the user's songs. Thus, the regularization model can be significantly improved, as it seems to be hesitant at choosing songs that don't match user songs, even with a low lambda value. Understanding the tradeoff between lambda values and the songs in the optimal playlist can help identify why this is the case. Unfortunately, plotting a tradeoff curve with this mixed integer program is unfeasible, and other methods to improve the regularization method must be analyzed in the future.

# Conclusion

The models for high and low energy situations, and when given user preferences, did an acceptable job at generating optimal playlists. This judgement is based on my personal analysis of the songs in the playlist. In order to better engage the accuracy of these playlists, more analysis from a myriad of listeners is needed. A larger survey of listeners on their opinions on these playlists would provide legitimacy to this model and these optimal playlists. The last model is useful for anyone, as it creates playlists based on user preferences, and should be appealing to any listener if they trust their own taste. If they don’t, they can decrease the lambda value and lower the requirement for the model to match your tastes. This will still create optimal playlists based on the user’s preferences, but to a lesser degree, and will allow the user to explore music similar to theirs that others are listening to. But more work still needs to be done to improve the regularization model, so that any changes in the lambda value are reflected in the songs in the optimal playlist. 

In the future, greater research and explanation could be used to identify which music characteristics correspond exactly to which type of playlists, which can increase the accuracy of the optimal playlists. I’d like to improve these models by adding more logical constraints that give the user a greater ability to design playlists of their liking. For instance, a music listener could want to restrict their playlists to specific genres or artists or change the length of the playlist. Unfortunately, the dataset used for this model lacks complete information on the genres of each song. Future data cleaning could add this as a variable in the dataset and allow this capability. 

Giving the users greater flexibility in designing the parameters of their playlist, or even allowing them to choose which music features they want to emphasize could allow this program to be more appealing to a larger audience. An app with all these features that emphasizes greater customization ability, combined with a seamless integration with Spotify, could be extremely useful for many music listeners. Imagine downloading an app that connects to Spotify, gathers your music data, and then automatically generates playlists for different situations directly into your Spotify app based on your preferences. I hope to create this in the near future, for all the music listeners out there who are too busy or lazy to create their own optimal playlists. 
    